Поймем, как примерно устроены декораторы `@classmethod` и `@staticmethod`.

In [10]:
class WithClassMethod():
    def __init__(self):
        pass

    def usual_method(self):
        print('usual:')
        print(self)

    def plain_class_method(arg):
        print(f'plain class {arg}')

    @classmethod
    def class_method(cls):
        print('class:')
        print(cls)

    @staticmethod
    def static_method():
        print('static')
        
wcm = WithClassMethod()

In [8]:
print(WithClassMethod.__dict__)

{'__module__': '__main__', '__init__': <function WithClassMethod.__init__ at 0x108a142f0>, 'usual_method': <function WithClassMethod.usual_method at 0x108a14378>, 'plain_class_method': <function WithClassMethod.plain_class_method at 0x108a140d0>, 'class_method': <classmethod object at 0x108a2d828>, 'static_method': <staticmethod object at 0x108a2d860>, '__dict__': <attribute '__dict__' of 'WithClassMethod' objects>, '__weakref__': <attribute '__weakref__' of 'WithClassMethod' objects>, '__doc__': None}


In [7]:
print(wcm.usual_method)
print(WithClassMethod.usual_method)
wcm.usual_method()
WithClassMethod.usual_method()

<bound method WithClassMethod.usual_method of <__main__.WithClassMethod object at 0x108a2d780>>
<function WithClassMethod.usual_method at 0x108a14378>
usual:


TypeError: usual_method() missing 1 required positional argument: 'self'

Видим, что для инстанса `usual_method` - это связанный метод, который находится в класс-объекте, и в который первым аргументом передается инстанс. А для класс-объекта это просто функция, которая лежит в его словаре. Поэтому вызвать ее без параметров мы не можем.

In [15]:
WithClassMethod.plain_class_method(10)
wcm.plain_class_method(10)

plain class 10


TypeError: plain_class_method() takes 1 positional argument but 2 were given

Раньше мы создавали статические методы класса, не указывая первым аргументом `self`. Теперь мы действительно не можем вызвать такой метод у инстанса с теми же аргументами, но он до сих пор связан с инстансом! И ничто не помешает нам передать в вызов на 1 аргумент меньше, таким образом, неявно отдав `self` первым аргументом и что-нибудь сломав внутри функции:

In [17]:
print(wcm.plain_class_method)
wcm.plain_class_method()

<bound method WithClassMethod.plain_class_method of <__main__.WithClassMethod object at 0x108a2f898>>
plain class <__main__.WithClassMethod object at 0x108a2f898>


Теперь посмотрим, что для нас сделает `@classmethod`:

In [20]:
print(WithClassMethod.class_method)
print(wcm.class_method)
WithClassMethod.class_method()
wcm.class_method()

<bound method WithClassMethod.class_method of <class '__main__.WithClassMethod'>>
<bound method WithClassMethod.class_method of <class '__main__.WithClassMethod'>>
class:
<class '__main__.WithClassMethod'>
class:
<class '__main__.WithClassMethod'>


Теперь наш метод `class_method` не просто функция в классе, а связанный с классом метод. Это означает, что при вызове такого метода первым аргументом туда неявно передается уже не инстанс, а сам класс-объект, причем поведение не зависит от того, вызвали мы этот метод у инстанса или у самого класс-объекта. Но что тогда лежит в словаре класса?

In [39]:
obj = WithClassMethod.__dict__['class_method']
print(obj)
for k, v in obj.__class__.__dict__.items():
    print(f'{k} : {v}')

__get__ : <slot wrapper '__get__' of 'classmethod' objects>
__init__ : <slot wrapper '__init__' of 'classmethod' objects>
__new__ : <built-in method __new__ of type object at 0x106242840>
__func__ : <member '__func__' of 'classmethod' objects>
__isabstractmethod__ : <attribute '__isabstractmethod__' of 'classmethod' objects>
__dict__ : <attribute '__dict__' of 'classmethod' objects>
__doc__ : classmethod(function) -> method

Convert a function to be a class method.

A class method receives the class as implicit first argument,
just like an instance method receives the instance.
To declare a class method, use this idiom:

  class C:
      @classmethod
      def f(cls, arg1, arg2, ...):
          ...

It can be called either on the class (e.g. C.f()) or on an instance
(e.g. C().f()).  The instance is ignored except for its class.
If a class method is called for a derived class, the derived class
object is passed as the implied first argument.

Class methods are different than C++ or Java

Оказывается, какой-то странный объект класса `classmethod`, который возвращает декоратор `@classmethod`. Здесь: https://docs.python.org/3/howto/descriptor.html#static-methods-and-class-methods есть немного подробностей, но если вкратце, то функция `classmethod()`  реализована на C, но для нее можно сконструировать python-аналог с помощью механизма дескрипторов. Она будет делать примерно следующее: принимать объект, и если он является классом, то передавать этот класс первым аргументом, а если он является чистым инстансом, то возьмет у него `type`.

Как написано выше в документации, `classmethod` это не статический метод в общепринятом смысле. Для устройства такого метода рекомендуют использовать `staticmethod`, который аналогичен `classmethod` за тем исключением, что не имеет неявных аргументов. Такой статический метод до сих пор может быть вызван как от класс-объекта, так и от инстанса, но не может конструировать инстансов класса внутри себя.

In [46]:
WithClassMethod.static_method()
wcm.static_method()

static
static


Отсутствие неявной передачи видно и по тому, что этот метод ни связан ни с каким объектом, а является просто некоторой функцией (хотя и до сих пор является объектом класса `staticmethod`):

In [45]:
print(WithClassMethod.static_method)
print(wcm.static_method)
print(WithClassMethod.__dict__['static_method'])

<function WithClassMethod.static_method at 0x108aa0b70>
<function WithClassMethod.static_method at 0x108aa0b70>


Теперь перенесем те же концепции на уровень метаклассов:

In [47]:
class Meta(type):
    X = [1, 2, 3]

    def some_method(cls):
        print(cls)
        return "foobar"

    @classmethod
    def bar_only(cls):
        print(cls)
        return "bar"

In [48]:
class Example(metaclass=Meta):
    def __init__(self):
        pass

In [49]:
print(Example.some_method)
print(Example.X)
print(Example.bar_only)

<bound method Meta.some_method of <class '__main__.Example'>>
[1, 2, 3]
<bound method Meta.bar_only of <class '__main__.Meta'>>


Теперь можно думать о класс-объекте `Example` как об инстансе, а метаклассе `Meta` как его классе. Тогда мы наблюдаем ту же самую картину - обычный метод `some_method` связан с инстансом, а метод класса `bar_only` связан с самим классом, которым в данном случае является `Meta`.

Можно заметить, что раньше обычные методы принимали первым аргументом `self`, а класс-методы `cls`, а теперь что-то как будто изменилось. На самом деле все осталось попрежнему, только в обычных методах мы поменяли имя `self` на `cls` в силу работы метакласса, для которого `self` на самом деле будет являться класс-объектом. Вспомним, что мы думаем об `Example` как об инстансе и посмотрим на такие вызовы:

In [54]:
Example.bar_only()
Example.some_method()
Meta.bar_only()
Meta.some_method()

<class '__main__.Meta'>
<class '__main__.Example'>
<class '__main__.Meta'>


TypeError: some_method() missing 1 required positional argument: 'cls'

Видно, что мы все так же не можем вызвать обычный метод у класс-объекта (в данном случае метакласса) без параметров, а в вызов у инстанса (в данном случае класс-объекта `Example`) происходит с неявной передачей этого инстанса.